# Simple OpenDrift Runner

Simple example of running OpenDrift with simulation output (nc-file) and randomly seeded particles as input.

** NOTE: **
The input nc-file must be written with *ignore_ghostcells* set to True, and *offset_x*- and *offset_y*-values set to produce valid and 'reasonable' lonlat-values for OpenDrift. Only tested with CTCS.

## Setup and initialization

In [ ]:
# Configuration
filename = 'netcdf_2018_02_12/CTCS_2018_02_12-16_43_12.nc' # netCDF-file with simulation timeseries
opendrift_directory = '/home/martinls/src/opendrift'

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
np.random.seed(983214)
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys
import glob
import math as m

import sys
from datetime import datetime

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, SimWriter, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

## Setup OpenDrift

In [ ]:
sys.path.append(opendrift_directory)

#from opendrift.models.leeway import Leeway
from opendrift.models.oceandrift import OceanDrift
#from opendrift.models.openoil3D import OpenOil3D

from opendrift.readers import reader_netCDF_CF_generic
#from opendrift.readers import reader_basemap_landmask

#from opendrift.readers.interpolation import ReaderBlock

print "Required input variables for drift model:"
print OceanDrift.required_variables
print "Available fallback values:"
print OceanDrift.fallback_values

## Print nc-file header and plot data

In [ ]:
import netCDF4 as nc

def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print "\t\ttype:", repr(nc_fid.variables[key].dtype)
            for ncattr in nc_fid.variables[key].ncattrs():
                print '\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr))
        except KeyError:
            print "\t\tWARNING: %s does not contain variable attributes" % key

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print "NetCDF Global Attributes:"
        for nc_attr in nc_attrs:
            print '\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print "NetCDF dimension information:"
        for dim in nc_dims:
            print "\tName:", dim 
            print "\t\tsize:", len(nc_fid.dimensions[dim])
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print "NetCDF variable information:"
        for var in nc_vars:
            if var not in nc_dims:
                print '\tName:', var
                print "\t\tdimensions:", nc_fid.variables[var].dimensions
                print "\t\tsize:", nc_fid.variables[var].size
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

In [ ]:
ncfile = nc.Dataset(filename, 'r')
nc_attrs, nc_dims, nc_vars = ncdump(ncfile, verb=False)

u = np.array(ncfile.variables['hu'])
v = np.array(ncfile.variables['hv'])
x = np.array(ncfile.variables['x'])
y = np.array(ncfile.variables['y'])
t = np.array(ncfile.variables['time'])

ncfile.close()

w = np.sqrt(u*u+v*v)

timestep = 1

#print 'timestep = ', t
#print 'x = ', x
#print 'y = ', y

plt.figure()
plt.imshow(w[timestep, :, :])
plt.colorbar()

## Initialize OpenDrift nc-reader and print reader information

In [ ]:
reader_arctic = reader_netCDF_CF_generic.Reader(filename)
reader_arctic.buffer = 10
reader_arctic.verticalbuffer = 0

# Separate reader for landmask
#reader_basemap = reader_basemap_landmask.Reader(
#                       llcrnrlon=5, llcrnrlat=55,
#                       urcrnrlon=15, urcrnrlat=65,
#                       resolution='h', projection='merc')

print reader_arctic

## Seed random-position particles

In [ ]:
def getCenter(reader):
    dims = np.array([reader.numx, reader.numy])
    center_xy = [reader.x[dims[0]/2], reader.y[dims[1]/2]]
    [lon, lat] = reader.xy2lonlat(center_xy[0], center_xy[1])
    return lon, lat

lon, lat = getCenter(reader_arctic)
print lon
print lat

In [ ]:
def sampleParticles(particles, radius_max, center_lat, center_lon):

    angle = np.random.rand(particles) * np.pi * 2
    radius = np.random.rand(particles)*radius_max

    lats = center_lat + np.sin(angle)*radius
    lons = center_lon + np.cos(angle)*radius
    
    return lats, lons

num_particles = 50
max_radius = 1
lat_array, lon_array = sampleParticles(num_particles, max_radius, lat, lon)

#num_particles = 2
#lat_array = [lat + 0.1, lat + 0.1]
#lon_array = [lon - 0.1, lon + 0.1]


plt.figure()
plt.plot(lat, lon, 'rx')
plt.plot(lat_array, lon_array, '.')

plot_axis = plt.axis()

## Advect the particles using velocity field imported from nc-file

In [ ]:
def simulateDrift(lon, lat, reader, start_time, end_time, reader_land_mask=None):
    o = OceanDrift(loglevel=20) # Output a minimum of information
    o.add_reader(reader) # with land mask from reader
    
    #o.add_reader([reader_land_mask, reader]) # with land mask from reader_land_mask
    
    #o.fallback_values['x_wind'] = 0
    #o.fallback_values['y_wind'] = 0
    #o.fallback_values['x_sea_water_velocity'] = 1.0
    #o.fallback_values['y_sea_water_velocity'] = 0.0
    #o.fallback_values['land_binary_mask'] = 0
    
    #o.set_config('processes:turbulentmixing', False)
    #o.set_config('general:coastline_action', 'none')

    print 'start time = ', start_time
    print 'end time = ', end_time
    num_timesteps = 100
    num_particles = len(lat)

    o.seed_elements(lon=lon, lat=lat, number=num_particles, radius=1, time=start_time)

    dt = (end_time - start_time).total_seconds() / num_timesteps
    o.run(end_time=end_time, time_step=dt, outfile='opendrift_output.nc')
    
    particleid=slice(None)
    timesteps=slice(None)
    lat_out=np.transpose(o.history['lat'][particleid, timesteps])
    lon_out=np.transpose(o.history['lon'][particleid, timesteps])
    
    print o
    #o.plot(background=['x_sea_water_velocity', 'y_sea_water_velocity'], buffer=.5)
    o.animation()
    o.plot()
    
    return lat_out, lon_out

    
lat_out, lon_out = simulateDrift(lon_array, lat_array, reader_arctic, reader_arctic.start_time, reader_arctic.end_time)#, reader_basemap)
print 'Timesteps, num_particles'
print lat_out.shape

plt.figure()
#plt.axis(plot_axis)
plt.plot(lat_array, lon_array, 'x')
plt.plot(lat_out, lon_out, '.')
plt.plot(lat, lon, 'd')